# Exercise 1: Introduction to tf.Variable
## Creating Variables:
Create a scalar variable.
Create a vector variable.
Create a matrix variable.

In [ ]:
import tensorflow as tf

In [ ]:
scalar_example = tf.Variable(2, name = 'scalar')
vector_example = tf.Variable([[1,2,3]], name = 'vector')
matrix_example = tf.Variable([[2,3,4],[4,5,6],[7,8,9]], name = 'matrix')

print(scalar_example)
print(vector_example)
print(matrix_example)

In [ ]:
# Print values of the variables
print("Scalar value:", scalar_example.numpy())
print("Vector values:", vector_example.numpy())
print("Matrix values:", matrix_example.numpy())

In [ ]:
scalar_example += 3.  # will not work
print(scalar_example)

In [ ]:
cost = tf.constant(4)
print(cost+4)

In [ ]:
# Update the variable by adding 3.0
scalar_example.assign_add(3)
print(scalar_example)
print(scalar_example.numpy())

In [ ]:
# Directly assign a new value to scalar variable

scalar_example.assign(10)
print(scalar_example.numpy())


In [ ]:
# Directly assign a new value to matrix variable
print(matrix_example.numpy())

matrix_example[0,1].assign(7)
print(matrix_example.numpy())

matrix_example[:,2].assign([0,40,60])
print(matrix_example.numpy())

In [ ]:
# Create two variables
var1 = tf.Variable(3.0, name='var1')
var2 = tf.Variable(4.0, name='var2')

# Perform a simple addition
result = var1 + var2

print("Result of addition:", result.numpy())

# Perform a multiplication
result = var1 * var2

print("Result of multiplication:", result.numpy())

# Introduction to Gradient Tape
In TensorFlow, the tf.GradientTape API records operations for automatic differentiation. It "watches" the operations executed within its context and builds a computation graph, which is then used to compute gradients of variables with respect to some loss or output.

In [ ]:
def f(w1, w2):
  return 3*w1**2 + 2*w1*w2

w1 = tf.Variable(5.0)
w2 = tf.Variable(6.0)

with tf.GradientTape() as tape:
  loss = f(w1,w2)

gradient = tape.gradient(loss, [w1,w2])
print(gradient)
del tape

In [ ]:
w1 = tf.Variable(5.0)
w2 = tf.Variable(6.0)

with tf.GradientTape(persistent = True) as tape:    # use persistnt = True
  loss = f(w1,w2)

gradient_w1 = tape.gradient(loss, w1)
gradient_w2 = tape.gradient(loss, w2)
print(gradient_w1)
print(gradient_w2)
del tape


By setting persistent=True in the tf.GradientTape constructor, the tape can be used to compute gradients multiple times. After you are done, you should delete the tape using del tape to release resources.

In [ ]:
input_variable = tf.Variable(initial_value=0.3)
with tf.GradientTape() as tape:
  result = tf.square(input_variable)
  gradient = tape.gradient(result, input_variable)
print(gradient)

In [ ]:
# Use of GradientTape with constant input tensor. By default graident tape will only track operations involving variables.
# if we try to compute gradients with regrad to anything else, it will generate None output. Hence use tape.watch

const1 = tf.constant(4.)

with tf.GradientTape() as tape:
  tape.watch(const1)
  result = tf.square(const1)
gradient = tape.gradient(result, const1)
print(gradient)



In [ ]:
# Use gradient tape with constant tensors

const1 = tf.constant(5.)
const2 = tf.constant(6.)

with tf.GradientTape() as tape:
  tape.watch(const1)
  tape.watch(const2)
  result = f(const1,const2)

gradient = tape.gradient(result, [const1,const2])
print(gradient)
del tape

In [ ]:
#Another way

const1 = tf.constant(5.)
const2 = tf.constant(6.)

with tf.GradientTape(persistent =True) as tape:
  tape.watch(const1)
  tape.watch(const2)
  result = f(const1,const2)

gradient_const1 = tape.gradient(result, const1)
gradient_const2 = tape.gradient(result, const2)
print(gradient_const1)
print(gradient_const2)
del tape

In [ ]:
# Working with Second-order gradients

time = tf.Variable(3.)
with tf.GradientTape() as outer:
  with tf.GradientTape() as inner:
    position = 5*time**2+7
    velocity = inner.gradient(position, time)
  acceleration = outer.gradient(velocity, time)
print(velocity)
print(acceleration)


In [ ]:
# Second-order gradients

time = tf.Variable(3.)
with tf.GradientTape(persistent = True) as outer:
    position = 5*time**2+7
    velocity = outer.gradient(position, time)
acceleration = outer.gradient(velocity, time)
print(velocity)
print(acceleration)

# Create Variables for Function
Define variables and a function
$$f(x,y) = (x-1)^2 + (y-1)^2 + xy$$
Use TensorFlow to find the minimum of the function
$f$ using gradient descent.

In [ ]:
x = tf.Variable(0.0, name = 'x')
y = tf.Variable(0.0, name = 'y')

def f_x(x,y):
  f1 = (x-1)**2+ (y-1)**2 + x*y
  return f1

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
epoch =  1000

for iter in range(epoch):
  with tf.GradientTape() as tape:
     loss = f_x(x,y)
  gradients = tape.gradient(loss, [x,y])
  optimizer.apply_gradients(zip(gradients, [x,y]))

print(f"Optimized parameters are x : {x.numpy()} and y: {y.numpy()}")




In [ ]:
x = tf.Variable(2.0)
y = tf.Variable(1.0)

# Not working this way
with tf.GradientTape(persistent = True) as tape:
  loss = f_x(x,y)
  [df_dx, df_dy] = tape.gradient(loss,[x, y])
  [d2f_dx2, d2f_dy2] = tape.gradient([df_dx, df_dy],[x, y])

print(df_dx, df_dy)
print(d2f_dx2, d2f_dy2)




In [ ]:
# This way working

with tf.GradientTape(persistent = True) as inne:
    loss = f_x(x,y)
    partial_derivative_x = inne.gradient(loss, x)
    partial_derivative_y = inne.gradient(loss, y)
    partial_derivative_xx = inne.gradient(partial_derivative_x, x)
    partial_derivative_xy = inne.gradient(partial_derivative_y, x)
    partial_derivative_yy = inne.gradient(partial_derivative_y, y)
    partial_derivative_yx = inne.gradient(partial_derivative_x, y)
print(partial_derivative_x)
print(partial_derivative_y)
print(partial_derivative_xx)
print(partial_derivative_yy)
print(partial_derivative_xy)
print(partial_derivative_yx)

In [ ]:
# Alternative way working

with tf.GradientTape(persistent = True) as tape:
  loss = f_x(x,y)
  [df_dx, df_dy] = tape.gradient(loss,[x, y])
  d2f_dx2 = tape.gradient(df_dx, x)
  d2f_dy2 = tape.gradient(df_dy, y)
  d2f_dydx = tape.gradient(df_dx, y)
  d2f_dxdy = tape.gradient(df_dy, x)

print("df/dx", df_dx)
print("df/dy", df_dy)
print("d2f/dx2", d2f_dx2)
print("d2f/dy2", d2f_dy2)
print("d2f/dydx",d2f_dydx)
print("d2f/dxdy",d2f_dxdy)

# Problem Statement

## Consider the following complex function:
$ f(x,y) = ae^{bx} \sin(cx) + b\cos(ax) + cx^{3}+ y^{2}$

where:
*   $a,b$  and $c$ are variables.
*   $x,y$ are constants

We will compute the gradients of $f$ with respect to $a, b,c$ at speciic value of $x, y$

In [ ]:
# Define constants

x = tf.constant(1.0)
y = tf.constant(2.0)

# Define Variables
a = tf.Variable(1.0)
b = tf.Variable(2.0)
c = tf.Variable(3.0)

# Define function
def cmplx_fn(a,b,c,x,y):
  #tape.watch([x,y])
  L = a*tf.exp(b*x)*tf.sin(c*x)+b*tf.cos(a*x)+ c*x**3 +y**2
  return L

with tf.GradientTape() as tape:
  loss = cmplx_fn(a,b,c,x,y)
  [d_a,d_b,d_c,d_x,d_y] =tape.gradient(loss, [a,b,c,x,y])
print(d_a, d_b, d_c, d_x, d_y)




In [ ]:
# In some situations we want to stop gradients from backpropagtaing through some part of the neural network
# tf.stop_gradient function returns its input during forward pass ( acts like tf.identity()) but it does not
#let grdaients through during backpropagation ( it acts like a constant)

def math_func(x,y):
  g = x*y
  g = tf.stop_gradient(g)
  return (x**2+y**2)*g

var1_tensor = tf.Variable(3.0)
var2_tensor = tf.Variable(4.0)

with tf.GradientTape() as tape:
  loss = math_func(var1_tensor, var2_tensor)

gardient = tape.gradient(loss, [var1_tensor, var2_tensor])
print(gardient)
